# Devoir TD sur le topic modeling
HO Xuan LO17 2021/2022


## 1. Importer des packages

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# NLTK Stop words
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import Phrases

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 2. Charger les données

In [2]:
#Load stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
df = pd.read_csv('data/EVENTS.csv')
df

,id,title,createdAt,updatedAt,startAt,endAt,enabled,lat,lng,fullAddress,link,url,body,authorId,authorType,authorZipCode
0,RXZlbnQ6MTc1NjE2ZDYtMWE2My0xMWU5LTk0ZDItZmExNj...,Réunion à la participation citoyenne pour le G...,2019-01-17 15:20:50,2019-02-25 17:07:49,2019-03-02 18:30:00,2019-03-02 23:00:00,True,43.620069,1.470152,"Maison de la Citoyenneté de Toulouse Est, Quar...",NaN,https://granddebat.fr/events/reunion-a-la-part...,<p><em>L’exactitude des informations ci-dessou...,VXNlcjo1MjY0ZDRhMi0xOTkxLTExZTktOTRkMi1mYTE2M2...,Élu / élue et Institution,31500
1,RXZlbnQ6MTc1NzUzMjUtMmJhMi0xMWU5LWJmNTYtZmExNj...,Grand débat des entreprises,2019-02-08 14:04:38,2019-02-08 14:04:38,2019-03-04 18:30:00,2019-03-04 22:00:00,True,44.988127,4.976960,"1, rue Marc Seguin, 26300 ALIXAN",NaN,https://granddebat.fr/events/grand-debat-des-e...,“<i>L’exactitude des informations ci-dessous r...,VXNlcjpiYjc3NDFkOC0yYWRiLTExZTktOTRkMi1mYTE2M2...,Élu / élue et Institution,26000
2,RXZlbnQ6MTc1OTk3MzYtMmY3ZC0xMWU5LWJmNTYtZmExNj...,Club du 3ème âge,2019-02-13 11:49:52,2019-02-13 11:49:51,2019-02-22 14:00:00,2019-02-22 17:30:00,True,49.211872,-0.378057,"Maison de Quartier, Place Dom Aubourg, 14000 CAEN",NaN,https://granddebat.fr/events/club-du-3eme-age,“<i>L’exactitude des informations ci-dessous r...,VXNlcjo2Y2NhNzhhOC0yZWE1LTExZTktYmY1Ni1mYTE2M2...,Organisation à but non lucratif,14000
3,RXZlbnQ6MTc1Y2E2YmMtMzQzOC0xMWU5LWJmNTYtZmExNj...,Grand débat national GAGNY,2019-02-19 12:18:32,2019-02-19 12:18:32,2019-02-25 19:00:00,2019-02-25 21:00:00,True,48.894931,2.538318,"Club Paul Éluard, 18 bis allée des Chênes, 932...",NaN,https://granddebat.fr/events/grand-debat-natio...,“<i>L’exactitude des informations ci-dessous r...,VXNlcjo4NmI2NzQ4OS0xYmNjLTExZTktOTRkMi1mYTE2M2...,Organisation à but non lucratif,93220
4,RXZlbnQ6MTc2NTVjZjYtMmJhMi0xMWU5LWJmNTYtZmExNj...,débat sur l'organisation de l'Etat et des serv...,2019-02-08 14:04:38,2019-02-08 14:04:38,2019-03-05 20:00:00,2019-03-05 22:00:00,True,47.122187,6.197913,"à la mairie de Saules, 25580 Saules",NaN,https://granddebat.fr/events/debat-sur-lorgani...,“<i>L’exactitude des informations ci-dessous r...,VXNlcjo0Y2I5MDc2ZS0yYWU1LTExZTktOTRkMi1mYTE2M2...,Élu / élue et Institution,25580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9970,RXZlbnQ6ZmZlNjA0OWUtM2U3Zi0xMWU5LWJmNTYtZmExNj...,Le Grand débat national des étudiants,2019-03-04 14:18:28,2019-03-04 14:18:28,2019-03-13 18:30:00,2019-03-13 20:00:00,True,51.037934,2.370704,"Centre de la Citadelle, 22 Avenue de l'Univers...",NaN,https://granddebat.fr/events/le-grand-debat-na...,“<i>L’exactitude des informations ci-dessous r...,VXNlcjo5NjVjMmU2Ni0xYjQ3LTExZTktOTRkMi1mYTE2M2...,NaN,59630
9971,RXZlbnQ6ZmZmNGE5MTctM2U3Zi0xMWU5LWJmNTYtZmExNj...,"Grand débat, réunion Assemblée générale CNATP ...",2019-03-04 14:18:28,2019-03-04 14:18:28,2019-03-06 19:00:00,2019-03-06 20:30:00,True,48.074954,-0.847880,"ZA Le CHATELLIER II, 13 RUE DU CHATELLIER, 599...",NaN,https://granddebat.fr/events/grand-debat-reuni...,“<i>L’exactitude des informations ci-dessous r...,VXNlcjpmMzI3N2QxMy0zZGQ0LTExZTktYmY1Ni1mYTE2M2...,Organisation à but non lucratif,53940
9972,RXZlbnQ6ZmZmNGY0MmQtMjMxNS0xMWU5LTk0ZDItZmExNj...,"""Pub and debate"" pour les français de Londres",2019-01-28 17:01:40,2019-01-28 17:01:39,2019-02-08 18:00:00,2019-02-08 21:00:00,True,51.588840,-0.403702,The case is altered - High Road - Eastcote Pin...,NaN,https://granddebat.fr/events/pub-and-debate-po...,<p><em>L’exactitude des informations ci-dessou...,VXNlcjo1YjAyYWNkOC0xOWJlLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,71880
9973,RXZlbnQ6ZmZmYzM3NTAtMzVjMS0xMWU5LWJmNTYtZmExNj...,Réunion publique Grand Débat National Citoyen ...,2019-02-21 11:18:15,2019-02-21 11:18:14,2019-03-08 19:00:00,2019-03-08 21:00:00,True,48.822549,2.361145,"11 rue Caillaux, 75013 Paris",https://docs.google.com/forms/d/e/1FAIpQLSe2IB...,https://granddebat.fr/events/reunion-publique-...,“<i>L’exactitude des informations ci-dessou

In [4]:
df.columns

Index(['id', 'title', 'createdAt', 'updatedAt', 'startAt', 'endAt', 'enabled',
       'lat', 'lng', 'fullAddress', 'link', 'url', 'body', 'authorId',
       'authorType', 'authorZipCode'],
      dtype='object')

In [5]:
df['body'][3]

"“<i>L’exactitude des informations ci-dessous relève uniquement de la responsabilité de l’organisateur de l’évènement“</i> <h2>Objet</h2> Permettre aux habitants de GAGNY de s'exprimer librement <h2>Public attendu</h2> Cet événement est ouvert au public dans la limite de 40 participants. L’inscription est requise. Pour participer, il suffit de vous adresser directement à l’organisateur de la réunion locale par courrier électronique : granddebatgagny@gmail.com"

In [6]:
data = df['body']

In [7]:
data

0       <p><em>L’exactitude des informations ci-dessou...
1       “<i>L’exactitude des informations ci-dessous r...
2       “<i>L’exactitude des informations ci-dessous r...
3       “<i>L’exactitude des informations ci-dessous r...
4       “<i>L’exactitude des informations ci-dessous r...
                              ...                        
9970    “<i>L’exactitude des informations ci-dessous r...
9971    “<i>L’exactitude des informations ci-dessous r...
9972    <p><em>L’exactitude des informations ci-dessou...
9973    “<i>L’exactitude des informations ci-dessous r...
9974    <p>“<i>L’exactitude des informations ci-dessou...
Name: body, Length: 9975, dtype: object

In [8]:
data.head()

0    <p><em>L’exactitude des informations ci-dessou...
1    “<i>L’exactitude des informations ci-dessous r...
2    “<i>L’exactitude des informations ci-dessous r...
3    “<i>L’exactitude des informations ci-dessous r...
4    “<i>L’exactitude des informations ci-dessous r...
Name: body, dtype: object

## 3. Nettoyer et vectoriser les documents

La liste complète des prétraitements :

1. Supprimer tous les HTML tags et la première citation
2. Supprimer tous les URL and email id,
3. Supprimer tous les stop-words, punctation et
4. Lemmatiser les mots restants et,
5. Mettre ces mots en minuscules.

In [9]:
nlp = spacy.load("fr_core_news_sm")

In [10]:
def delete_NaN_lines(input_data):
    remove_nan = input_data.dropna()
    print(type(remove_nan))
    return remove_nan
textes = delete_NaN_lines(data)
print(type(textes))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [11]:
def delete_tag(input_data):
    result = []
    for element in input_data:
        temp  = element.replace("L’exactitude des informations ci-dessous relève uniquement de la responsabilité de l’organisateur de l’évènement","")
        temp = temp.replace('<i>',"")
        temp = temp.replace('</i>',"")
        temp = temp.replace('<h1>',"")
        temp = temp.replace('</h1>',"")
        temp = temp.replace('<h2>',"")
        temp = temp.replace('</h2>',"")
        temp = temp.replace('<h3>',"")
        temp = temp.replace('</h3>',"")
        temp = temp.replace('<p>',"")
        temp = temp.replace('</p>',"")
        temp = temp.replace('<em>',"")
        temp = temp.replace('</em>',"")
        temp = temp.replace('&nbsp;',"")
        result.append(temp)
    return result

In [12]:
delete_textes = delete_tag(textes)
textes = pd.Series(delete_textes)
textes

0       L’exactitude des informations ci-dessous relèv...
1       ““ Objet Faire émerger les propositions des en...
2       ““ Objet Ce grand débat est l'occasion de fair...
3       ““ Objet Permettre aux habitants de GAGNY de s...
4       ““ Objet débattre avec les habitants de la com...
                              ...                        
9968    ““ Objet Les étudiants de l'ULCO de Dunkerque ...
9969    ““ Objet Permettre aux entreprises artisanales...
9970    L’exactitude des informations ci-dessous relèv...
9971    ““ Objet Nous souhaitons faire participer les ...
9972    ““  Objet  Dans le cadre du grand débat nation...
Length: 9973, dtype: object

In [13]:
textes[0]

"L’exactitude des informations ci-dessous relève uniquement de la responsabilité de l’organisateur de l’événement.  Objet  Le collectif «Groupe Vox Populi, l'association Occitanie Communication et le Collectif Interdépartemental de Défense de l'Usager Citoyen Contribuable» organise une réunion en vue du Grand Débat. Venez nombreuses et nombreux faire part de vos sollicitations.  Public attendu  Cet événement est ouvert au public dans la limite de 120 participants. L'inscription n'est pas requise : pour participer, rendez-vous à l'heure dite et au lieu indiqué."

In [14]:
%%time

spacy_docs = list(nlp.pipe(textes))

CPU times: total: 1min 25s
Wall time: 1min 35s


In [15]:
print(type(spacy_docs))
print(len(spacy_docs))

<class 'list'>
9973


In [16]:
spacy_docs[1]

““ Objet Faire émerger les propositions des entrepreneurs de la Drôme (format 3 ateliers + réunion plénière) Public attendu Cet événement n’est pas ouvert au public

In [17]:
%%time
import re

docs     = []

for doc in spacy_docs:
    
    tokens = []
    for token in doc:
            if not token.like_email and not token.like_url and not token.is_stop and not token.is_punct : 
                tokens.append(token.lemma_.lower())
    docs.append( tokens )

CPU times: total: 484 ms
Wall time: 569 ms


Voici un aperçu du **premier** document tokenisé :

In [18]:
print(docs[1])

['objet', 'faire', 'émerger', 'proposition', 'entrepreneur', 'drôme', 'format', '3', 'atelier', 'plus', 'réunion', 'plénier', 'public', 'événement', 'public']


**deuxième** document tokenisé :

In [19]:
print(docs[2])

['objet', 'grand', 'débat', 'occasion', 'faire', 'exprimer', 'personne', 'âgé', 'quartier', 'contribution', 'participer', 'recherche', 'solution', 'face', 'difficulté', 'pays', 'rencontr', "aujourd'hui", 'moyen', 'population', 'rester', 'actif', 'intellectuellemer', 'public', 'événement', 'public']


In [20]:
%%time

docs = []
for doc in spacy_docs:
    tokens = []
    for token in doc:
        if len(token.orth_) > 3 and not token.is_stop:
            token = token.lemma_.lower()
            tokens.append(token)
    docs.append( tokens )

CPU times: total: 719 ms
Wall time: 814 ms


In [21]:
print(docs[0])

['exactitude', 'information', 'ci-dessous', 'relever', 'uniquement', 'responsabilité', 'organisateur', 'événement', 'objet', 'collectif', 'groupe', 'populi', 'association', 'occitanie', 'communication', 'collectif', 'interdépartemental', 'défense', 'usager', 'citoyen', 'contribuable', 'organise', 'réunion', 'grand', 'débat', 'venez', 'faire', 'part', 'sollicitation', 'public', 'événement', 'public', 'limite', 'participant', 'inscription', 'requérir', 'participer', 'rendez-vous', 'heure', 'lieu', 'indiquer']


In [22]:
print(docs[1])

['objet', 'faire', 'émerger', 'proposition', 'entrepreneur', 'drôme', 'format', 'atelier', 'réunion', 'plénier', 'public', 'événement', 'public']


In [23]:
bigram = Phrases(docs, min_count=10)
i = 0
for index in range(len(docs)):
    for token in bigram[docs[index]]:
        if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
            docs[index].append(token)

In [24]:
print(docs[4])

['objet', 'débattre', 'habitant', 'commune', 'organisation', 'état', 'service', 'public', 'public', 'événement', 'public', 'inscription', 'participer', 'rendez-vous', 'heure', 'lieu', 'indiquer', 'habitant_commune', 'organisation_état', 'rendez-vous_heure', 'lieu_indiquer']


In [25]:
dictionary = Dictionary(docs)
print('Nombre de mots unique dans les documents initiaux :', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.25)
print('Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents :', len(dictionary))

print("Exemple :", dictionary.doc2bow(docs[4]))

Nombre de mots unique dans les documents initiaux : 12256
Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents : 4210
Exemple : [(50, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1)]


In [26]:
corpus = [dictionary.doc2bow(docs[i]) for i in range(len(docs))]
corpus

[[(0, 1),
  (1, 1),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1)],
 [(6, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1)],
 [(6, 1),
  (8, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1)],
 [(14, 1),
  (30, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1)],
 [(50, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1)],
 [(50, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1)],
 [],
 [(1, 1),
  (6, 1),
  (14, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1)],
 [(6, 1),
  (8, 1),
  (20, 1),
  (24, 1),
  (78, 

## 4. Topic Modeling avec LDA

Maintenant, il est temps d'entraîner notre LDA ! Pour ce faire, nous utilisons les paramètres suivants : 

- **corpus** : les représentations en sac-de-mots de nos documents
- **id2token** : le mappage des index aux mots
- **num_topics** : le nombre de topics que le modèle doit identifier (fixons à <font color = "red"><b>10</b></font>)
- **chunksize** : le nombre de documents que le modèle voit à chaque mise à jour (fixons à <font color = "red"><b>1 000</b></font>)
- **passes** : le nombre de fois où nous montrons le corpus total au modèle pendant l'entraînement (fixons à <font color = "red"><b>5</b></font>)
- **random_state** : nous utilisons une graine pour assurer la reproductibilité (fixons à <font color = "red"><b>1</b></font>)

Remarquons que l'entraînement peut durer quelques minutes

In [27]:
%%time

model = LdaModel(corpus,id2word = dictionary,num_topics=10,chunksize=1000,passes=5,random_state=1)

CPU times: total: 17.6 s
Wall time: 20.5 s


In [28]:
for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.042*"thème" + 0.037*"grand_débat" + 0.035*"démocratie" + 0.029*"fiscalité" + 0.029*"organisation" + 0.026*"national" + 0.025*"service" + 0.024*"dépense" + 0.020*"fiscalité_dépense" + 0.020*"citoyenneté"

***********
* topic 2 *
***********
2 : 0.021*"français" + 0.018*"faire" + 0.013*"nécessit" + 0.013*"citoyen" + 0.011*"mieux" + 0.011*"pays" + 0.011*"débattre" + 0.010*"proposition" + 0.010*"politique" + 0.010*"démocratique"

***********
* topic 3 *
***********
3 : 0.051*"national" + 0.045*"cadre" + 0.038*"débat_national" + 0.037*"cadre_grand" + 0.021*"organiser" + 0.019*"parole" + 0.017*"citoyen" + 0.014*"donner" + 0.014*"grand_débat" + 0.014*"donner_parole"

***********
* topic 4 *
***********
4 : 0.084*"local" + 0.075*"réunion_local" + 0.075*"requérir" + 0.074*"adresser" + 0.073*"courrier" + 0.072*"électronique" + 0.072*"courrier_électronique" + 0.070*"adresser_organisateur" + 0.057*"inscrire" + 0.042*"requérir_inscrire"

***********
* topic

In [29]:
for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.042*"thème" + 0.037*"grand_débat" + 0.035*"démocratie" + 0.029*"fiscalité" + 0.029*"organisation" + 0.026*"national" + 0.025*"service" + 0.024*"dépense" + 0.020*"fiscalité_dépense" + 0.020*"citoyenneté"

***********
* topic 2 *
***********
2 : 0.021*"français" + 0.018*"faire" + 0.013*"nécessit" + 0.013*"citoyen" + 0.011*"mieux" + 0.011*"pays" + 0.011*"débattre" + 0.010*"proposition" + 0.010*"politique" + 0.010*"démocratique"

***********
* topic 3 *
***********
3 : 0.051*"national" + 0.045*"cadre" + 0.038*"débat_national" + 0.037*"cadre_grand" + 0.021*"organiser" + 0.019*"parole" + 0.017*"citoyen" + 0.014*"donner" + 0.014*"grand_débat" + 0.014*"donner_parole"

***********
* topic 4 *
***********
4 : 0.084*"local" + 0.075*"réunion_local" + 0.075*"requérir" + 0.074*"adresser" + 0.073*"courrier" + 0.072*"électronique" + 0.072*"courrier_électronique" + 0.070*"adresser_organisateur" + 0.057*"inscrire" + 0.042*"requérir_inscrire"

***********
* topic

In [30]:
pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)

C:\Users\Ho Xuan Vinh\AppData\Roaming\Python\Python39\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.071702 -0.054817       1        1  13.476024
1      0.124423  0.070747       2        1   9.438610
2      0.041560 -0.218774       3        1  13.374229
3     -0.331529  0.031169       4        1   8.903150
4      0.129726  0.016782       5        1   8.115905
5     -0.308168  0.072582       6        1   7.559212
6      0.126848  0.318266       7        1   7.135281
7      0.013440 -0.139736       8        1  16.132433
8      0.085360  0.019068       9        1   9.410582
9      0.046637 -0.115287      10        1   6.454574, topic_info=              Term         Freq        Total Category  logprob  loglift
14        requérir  1849.000000  1849.000000  Default  30.0000  30.0000
55   réunion_local  1055.000000  1055.000000  Default  29.0000  29.0000
44        adresser  1046.000000  1046.000000  Default  28.0000  28.0000
46        courrier  1024.000000  1024.000000  Default  27.0000  27.0000
56    électronique  1019.000000  1019.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
74        inscrire    88.388204  2063.982638  Topic10  -4.7475  -0.4103
139     initiative    69.596714   423.041069  Topic10  -4.9866   0.9356
120       question    70.725130   708.247531  Topic10  -4.9705   0.4364
180        pouvoir    65.288117   874.130240  Topic10  -5.0505   0.1460
70          parole    64.619001   733.583535  Topic10  -5.0608   0.3109

[614 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
642       8  0.978269                      
3231      2  0.964829                 /span
1258      1  0.246938          100personnes
1258      4  0.740814          100personnes
1259      1  0.249712  100personnes_maximum
...     ...       ...                   ...
393       3  0.076721                  être
393       5  0.043841                  être
393       7  0.115082                  être
393       8  0.005480                  être
393       9  0.246605                  être

[1178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [31]:
n_doc = 4
i = 0
for (text, doc) in zip(textes[:n_doc], docs[:n_doc]):
    i += 1
    print("***********")
    print("* doc", i, "  *")
    print("***********")
    print(text)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.1])
    print()

***********
* doc 1   *
***********
L’exactitude des informations ci-dessous relève uniquement de la responsabilité de l’organisateur de l’événement.  Objet  Le collectif «Groupe Vox Populi, l'association Occitanie Communication et le Collectif Interdépartemental de Défense de l'Usager Citoyen Contribuable» organise une réunion en vue du Grand Débat. Venez nombreuses et nombreux faire part de vos sollicitations.  Public attendu  Cet événement est ouvert au public dans la limite de 120 participants. L'inscription n'est pas requise : pour participer, rendez-vous à l'heure dite et au lieu indiqué.
[(9, 0.60717744), (10, 0.27230376)]

***********
* doc 2   *
***********
““ Objet Faire émerger les propositions des entrepreneurs de la Drôme (format 3 ateliers + réunion plénière) Public attendu Cet événement n’est pas ouvert au public
[(2, 0.3224195), (3, 0.12737368), (8, 0.26945886), (10, 0.22067441)]

***********
* doc 3   *
***********
““ Objet Ce grand débat est l'occasion de faire s'expr